# Simple Feature Based Scheduler run with SOCS

This example notebook show how to do a simple 1 day FBS run using SOCS. In this example we use the default FBS configuration, a separate example will be given on how to provide a custom configuration. 

Before running the notebook make sure you run `manage_db --save-dir $HOME/run_local/output/` on the command line to setup the SOCS database. 

In [ ]:
import logging
import healpy as hp

from lsst.sims.ocs.database import SocsDatabase
from lsst.sims.ocs.kernel import Simulator
from lsst.sims.featureScheduler.driver import FeatureSchedulerDriver as Driver
from lsst.sims.ocs.setup import create_parser
from lsst.sims.ocs.setup import apply_file_config, read_file_config

In [ ]:
logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M')

This next cell loads default command line arguments. These are needed mainly to setup the database.

In [ ]:
parser = create_parser()
args = parser.parse_known_args()[0]
prog_conf = read_file_config()
if prog_conf is not None:
    apply_file_config(prog_conf, args)
print(args.sqlite_save_dir, args.session_id_start, args.sqlite_session_save_dir)

Setup socs database to store simulations results, if needed. 

In [ ]:
db = SocsDatabase(sqlite_save_path=args.sqlite_save_dir,
                  session_id_start=args.session_id_start,
                  sqlite_session_save_path=args.sqlite_session_save_dir)

In [ ]:
session_id = db.new_session("FBS test on notebook")

We now define a driver for the simulation. In this case, we already imported the FBS driver as Driver so we simply call it. 

In [ ]:
driver = Driver()

By default the duration of a simulation is 10 years. Here we will run a single day. 

In [ ]:
args.frac_duration = 0.003

We now set the SOCS simulator

In [ ]:
sim = Simulator(args, db, driver=driver)

In [ ]:
sim.initialize()

And run the simulation

In [ ]:
sim.run()

We now have access to all the scheduler data structure to play with. In the cell bellow, we plot the TargetMapBasis function for the `g` filter.

In [ ]:
hp.mollview(sim.driver.scheduler.survey_lists[0][1].basis_functions[2]())